# D-Hydro Boezemmodel V4

Met deze notebook bouw je een boezemmodel voor Nooderzijlvest. 

De notebook is opgezet door Daniel Tollenaar (D2Hydro), Vincent de Looij (Waterschap Noorderzijlvest) en Siebe Bosch (HydroConsult). Het Notebook is gebaseerd op een  <a href="https://github.com/openearth/delft3dfmpy/blob/master/notebooks/Usage_introduction_FM_only.ipynb">voorbeeldnotebook</a> uit D-HyDAMO, opgezet door HKV Lijn in Water: 


## Installatie
Zorg dat je beschikt over een werkende Python installatie en <a href="https://github.com/openearth/delft3dfmpy#installation">D-HYDAMO omgeving</a>. 

Het resulterende model moet geimporteerd kunnen worden in D-HYDRO versie 0.9.7.52006 of hoger.

Download de bestanden van <a href="https://drive.google.com/drive/folders/1BXNEwlEDCtShDj5KVuHQcukup-QLTuzR">Google Drive</a> en zet deze in de folder .\data

De verwijzing naar bestanden is geschreven in het Nederlands. De rest van de code is geschreven in het Engels. Elk code-blok is voorzien van uitleg boven het desbetreffende blok, geschreven in het Nederlands.

<b>let op:</b> deze notebook werkt alleen in combinatie met een config.py. In config.py dient een variabele dimr_path te zijn opgegeven, afhankelijk van de D-Hydro installatie. Bijvoorbeeld:

dimr_path = r"c:\Program Files (x86)\Deltares\D-HYDRO Suite 1D2D (Beta) (0.9.7.52006)\plugins\DeltaShell.Dimr\kernels\x64\dimr\scripts\run_dimr.bat"

## Controle bestanden

Alle bestanden die gebruikt worden in deze tutorial staan in het code-blok hieronder. Wanneer je dit codeblok uitvoert wordt de aanwezigheid van deze bestanden gecontroleerd.



In [1]:
from pathlib import Path
from config import dimr_path
import pandas as pd
import numpy as np
import rasterio
from rasterstats import zonal_stats

data_path = Path(r".\data").absolute().resolve()
beheerregister = data_path.joinpath("beheerregister")
excelbestanden = data_path.joinpath("xlsx")
bathymetrie = data_path.joinpath("bathymetrie")

modelbestanden = {"randvoorwaarden":"randvoorwaarden.xlsx",
                  "uitgesloten_profielen":"uitgesloten_profielen.xlsx"}

invoerbestanden = {
    "modelgebied": "Bemalings_en_lozingsgebieden.shp",
    "branches": "Afaanvoervakken_DHYDRO_v5.shp",
    "profielpunten": "profielpunten_3feb2021_actueel.shp",
    "bruggen": "Bruggen_(doorstroomopeningen).shp",
    "duikers": "duikers.shp",
    "sluizen": "sluizen.shp",
    "sifons": "sifons.shp",
    "stuwen": "stuwen.shp",
    "inlaten": "inlaten.shp",
    "gemalen": "gemalen_v2.shp",
    "peilgebieden": "Peilgebieden.shp"
}

bathymetriebestanden = {
    "hoogte_raster": "AHN3_Lauwersmeer_gevuld.tif",
    "zones": "Polygonen_Lauwersmeer.shp"
}

vorm_mapping = dict(rond=1,
                    driehoekig=1,
                    rechthoekig=3,
                    eivormig=1,
                    ellips=1,
                    Paraboolvormig=1,
                    trapeziumvormig=1,
                    heul=1,
                    muil=3,
                    langwerpig=1,
                    scherp=1,
                    onbekend=99,
                    overig=99)

ruwheid_mapping = {"A1": 0.03,
                   "A2": 0.03,
                   "A2 Boot": 0.03,
                   "A3": 0.03,
                   "B1": 0.02,
                   "B2": 0.022,
                   "C1": 0.02,
                   "C2b": 0.022,
                   "DERDEN": 0.03,
                   "GEEN": 0.07}

# richting_mapping = {"schutsluis naar een zijde": "positive",
#                    "overig": "none",
#                    "schutsluis naar twee zijden": "both",
#                    "uitwateringssluis/spuisluis": "positive"}

richting_mapping = {"schutsluis naar een zijde": "none",
                    "overig": "both",
                    "schutsluis naar twee zijden": "none",
                    "uitwateringssluis/spuisluis": "positive"}

for key, item in invoerbestanden.items():
    if not beheerregister.joinpath(item).exists():
        print(f"bestand voor {key} bestaat niet: {beheerregister.joinpath(item)}")
        
for key, item in bathymetriebestanden.items():
    if not bathymetrie.joinpath(item).exists():
        print(f"bestand voor {key} bestaat niet: {bathymetrie.joinpath(item)}")
        
for key, item in modelbestanden.items():
    if not excelbestanden.joinpath(item).exists():
        print(f"bestand voor {key} bestaat niet: {excelbestanden.joinpath(item)}")

rekenpunt_afstand = 250.0


## Inlezen beheerregister in HyDAMO

### Aanmaken HyDAMO object

Alle benodigde modules worden geimporteerd en het HYDAMO object wordt aangemaakt bij het uitvoeren van onderstaand code-blok

In [2]:
from delft3dfmpy import DFlowFMModel, HyDAMO, Rectangular, DFlowFMWriter
from delft3dfmpy import DFlowRRModel, DFlowRRWriter
from delft3dfmpy.datamodels.common import ExtendedGeoDataFrame
import hydrotools
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString

hydamo = HyDAMO(extent_file=str(beheerregister.joinpath(invoerbestanden["modelgebied"])))

### hulp-variabelen

#### snap_to_branches:
De optie om bij het inlezen te snappen naar branches en alles dat niet op boezemmodel_v4 ligt weg te gooien. Wordt nu alleen gebruikt bij duikers

#### kwk_groups_df:
De kunstwerkgroepen waarmee we compound-structures genereren

In [3]:
branches = gpd.read_file(beheerregister.joinpath(invoerbestanden["branches"]))
branches.set_index("OVKIDENT", inplace=True)
branches.columns = branches.columns.str.lower()
snap_to_branches = dict(branches=branches,
                        distance=1,
                        attribute_filter={"hydromodel": "Boezemmodel_v4"}) 

kwk_groups_df = pd.read_csv(excelbestanden.joinpath("kunstwerkgroepen.csv"), index_col="code")

### Toevoegen waterlopen

De waterlopen (branches) worden toegevoegd met volgend code-blok:
* de branches worden toegekend aan het hydamo-object
* een HyDAMO ruwheidscode (2 = manning) wordt toegekend
* De ruwheidswaarde wordt bepaalt aan de hand van de werkcode
* de eindpunten van de branches worden gesnapped binnen een tolerantie van 1m. De coordinaten worden ook afgerond op 1m.

In [4]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["branches"]),
                           "branches",
                           attribute_filter={"hydromodel": "Boezemmodel_v4"},
                           keep_columns=["HydroModel",
                                         "bodemhoogtebovenstrooms",
                                         "bodembreedte",
                                         "taludhellinglinkerzijde",
                                         "taludhellingrechterzijde",
                                         "IWS_W_WATB",
                                         "WERKCODE"],
                           column_mapping={"OVKIDENT": "code",
                                           "IWS_AVVHOB": "bodemhoogtebovenstrooms",
                                           "AVVBODDR": "bodembreedte",
                                           "AVVTALUL": "taludhellinglinkerzijde",
                                           "AVVTALUR": "taludhellingrechterzijde"}
                          )

hydamo.branches.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.branches["ruwheidstypecode"] = 2
hydamo.branches["ruwheidswaarde"] = hydamo.branches.apply((lambda x: ruwheid_mapping[x["werkcode"]]), axis=1)

hydamo.branches = hydrotools.snap_ends(hydamo.branches, tolerance=1, digits=1)

### Aanmaken principeprofielen
Als terugvaloptie voor branches zonder profielen schrijven we principe profielen weg die we later in deze notebook gebruiken.

In [5]:
principe_profielen_df = hydrotools.get_trapeziums(gdf,
                                                  "code",
                                                  "bodembreedte",
                                                  "bodemhoogtebovenstrooms",
                                                  "iws_w_watb",
                                                  "taludhellinglinkerzijde",
                                                  "taludhellingrechterzijde")

principe_profielen_df.to_csv(excelbestanden.joinpath("principe_profielen.csv"))

### Toevoegen yz-profielen
De yz-profielen worden ingeladen:
* De profielpunten worden geopend
* de punten worden geordend en geconverteerd naar polylinen met een xyz coordinaten
* lijnen langer dan 500m worden weggegooid, omdat er soms kademuren PRO_TYPE = PRO hebben gekregen
* lijnen worden toegekend aan het HyDAMO object
* lijnen die niet snappen met de branches, worden weggegooid
* lijnen krijgen de ruwheidswaarde mee van de onderliggende tak (branch)

In [6]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["profielpunten"]),
                           "crosssections",
                           attribute_filter={"osmomsch_1": ["Z1"],
                                             "PRO_TYPE": "PRO"},
                           column_mapping={"PROIDENT": "code",
                                           "IWS_VOLGNR": "order",
                                           "OSMOMSCH_1": "category",
                                           "iws_hoogte": "z"},
                           z_coord=True
                           )

grouper = gdf.groupby("code")
profiles = dict()

for code, prof_gdf in grouper:
    prof_gdf = prof_gdf.sort_values("order")  #Staan ook punten met zelfde order
    first_point = prof_gdf.iloc[0]["geometry"]
    cum_dist = -1
    line = []
    for idx, (_, row) in enumerate(prof_gdf.iterrows()):
        geom = row["geometry"]
        distance = geom.distance(first_point)
        if distance > cum_dist:
            cum_dist = distance
            line += [(geom.x, geom.y, row["z"])]

    profiles[code] = [code, LineString(line)]

profiles_gdf = gpd.GeoDataFrame.from_dict(profiles,
                                          orient="index",
                                          columns=["code",
                                                   "geometry"]
                                          )

profiles_gdf["ruwheidstypecode"] = 2

profiles_gdf["codegerelateerdobject"] = None

profiles_gdf = profiles_gdf.loc[profiles_gdf["geometry"].length < 500]

exclude_xlsx = excelbestanden.joinpath(modelbestanden["uitgesloten_profielen"])
exclude_list = pd.read_excel(exclude_xlsx, header=None)[0].to_list()
profiles_gdf = profiles_gdf.loc[~profiles_gdf.index.isin(exclude_list)]

hydamo.crosssections.set_data(profiles_gdf,
                              index_col="code",
                              check_columns=False,
                              check_geotype=True)

hydamo.crosssections.snap_to_branch(hydamo.branches, snap_method="intersecting")
hydamo.crosssections.dropna(axis=0, inplace=True, subset=["branch_offset"])
hydamo.crosssections["ruwheidswaarde"] = hydamo.crosssections.apply(
    (lambda x: hydamo.branches.loc[x["branch_id"]]["ruwheidswaarde"]), axis=1)


### Toevoegen geparameteriseerde profielen

Per brug moet er een profiel worden toegevoegd

In [7]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["bruggen"]),
                           "parametrised_profiles",
                           column_mapping={"bodemh_bov": "bodemhoogtebovenstrooms",
                                           "bodemh_ben": "bodemhoogtebenedenstrooms",
                                           "doorstroom": "bodembreedte",
                                           "hoogte_ond": "hoogteinsteeklinkerzijde",
                                           })

grouper = gdf.groupby("code")
data = {item: [] for item in ["code",
                              "bodemhoogtebovenstrooms",
                              "bodemhoogtebenedenstrooms",
                              "bodembreedte",
                              "hoogteinsteeklinkerzijde",
                              "geometry",
                              "codegerelateerdobject"]}

for code, df in grouper:
    data["code"] += [f"PRO_{code}"]
    data["codegerelateerdobject"] += [code]
    for item in ["bodemhoogtebovenstrooms",
                 "bodemhoogtebenedenstrooms",
                 "hoogteinsteeklinkerzijde",
                 "geometry"]:
        data[item] += [df.iloc[0][item]]
    data["bodembreedte"] += [df['bodembreedte'].sum()]

gdf = gpd.GeoDataFrame(data)
gdf["geometry"] = gdf.apply((lambda x: LineString([[x["geometry"].x-0.1,
                                                    x["geometry"].y-0.1],
                                                  [x["geometry"].x+0.1,
                                                   x["geometry"].y+0.1]])),
                            axis=1)

gdf["hoogteinsteekrechterzijde"] = gdf["hoogteinsteeklinkerzijde"]
gdf["ruwheidswaarde"] = 15
gdf["ruwheidstypecode"] = 4
gdf["taludhellinglinkerzijde"] = 1
gdf["taludhellingrechterzijde"] = 1

hydamo.parametrised_profiles.set_data(gdf,
                                      index_col="code",
                                      check_columns=True,
                                      check_geotype=False)

### Toevoegen gemalen

ToDo:
- @Vincent: per gemaal de bovenstroomse GPG specificeren i.v.m. bepalen aan- afslagpeilen

In [8]:
#we lezen hier al de GPG's in, omdat we alleen gemalen toevoegen waarvan we een peil hebben
gpg_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["peilgebieden"]),
                                 "sturing",
                                 column_mapping={"GPGIDENT": "code",
                                                 "OPVAFWZP": "streefwaarde"
                                                 })

gpg_gdf["codegerelateerdobject"] = [code.replace("GPG", "") for code in gpg_gdf["code"]]

pumps_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["gemalen"]),
                                 "pumps",
                                 attribute_filter={"functie": "afvoergemaal"},
                                 column_mapping={"gecom_max_": "maximalecapaciteit"})

pumps_gdf = pumps_gdf.loc[pumps_gdf["code"].isin(gpg_gdf["codegerelateerdobject"])]

gemalen_gdf = pumps_gdf.copy()
pumps_gdf["codegerelateerdobject"] = pumps_gdf["code"].copy()
pumps_gdf["code"] = [f"PMP_{code}" for code in pumps_gdf["code"]]
hydamo.gemalen.set_data(gemalen_gdf,
                        index_col="code",
                        check_columns=True,
                        check_geotype=True)
hydamo.gemalen.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=5)
hydamo.gemalen.dropna(axis=0, inplace=True, subset=["branch_offset"])
hydamo.pumps.set_data(pumps_gdf,
                      index_col="code",
                      check_columns=True,
                      check_geotype=True)
hydamo.pumps.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=1)
hydamo.pumps.dropna(axis=0, inplace=True, subset=["branch_offset"])

print(len(hydamo.pumps))

90


### Toevoegen bruggen
Per brug moet er een profiel worden toegevoegd

In [9]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["bruggen"]),
                           "bridges",
                           attribute_filter={"KBRSOORT": [2, 3, 4]},
                           snap_to_branches=snap_to_branches,
                           column_mapping={"doorstro_2": "lengte",
                                           "bodemh_bov": "hoogteonderzijde"})

grouper = gdf.groupby("code")
data = {"code": [code for code, frame in grouper],
        "hoogteonderzijde": [frame["hoogteonderzijde"].values[0] for code,
                             frame in grouper],
        "lengte": [frame["lengte"].values[0] for code, frame in grouper],
        "geometry": [frame["geometry"].values[0] for code, frame in grouper]
        }
gdf = gpd.GeoDataFrame(data)

gdf["hoogtebovenzijde"] = gdf["hoogteonderzijde"] + 1

gdf["dwarsprofielcode"] = gdf["code"].apply(lambda x: f"PRO_{x}")
gdf["intreeverlies"] = 0.7
gdf["uittreeverlies"] = 0.7
gdf["ruwheidstypecode"] = 4
gdf["ruwheidswaarde"] = 70

hydamo.bridges.set_data(gdf, index_col="code", check_columns=True, check_geotype=False)
hydamo.bridges.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=5)
hydamo.bridges.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Aanmaken sluizen (als duiker)

Sluizen worden toegevoegd als hoge duikers (10m). Ontbrekende waarden worden als volgt ingevuld:
- Aantal kokers = 1
- Lengte = 20m
- Breedte = breedte principe profiel onderliggende branch
- BOK = hoogte principe profiel onderliggende branch

In [10]:
sluices_gdf = hydrotools.read_file(
    beheerregister.joinpath(invoerbestanden["sluizen"]),
    "culverts",
    attribute_filter={"SOORT": ["schutsluis naar een zijde",
                                "overig",
                                "schutsluis naar twee zijden",
                                "uitwateringssluis/spuisluis"]},
    keep_columns=["AANTAL_SLU", "SOORT", "branch_id"],
    column_mapping={
        "KOLKWIJDTE": "breedteopening",
        "BOS_BENEDE": "hoogtebinnenonderkantbenedenstrooms",
        "BOS_BOVEN": "hoogtebinnenonderkantbovenstrooms",
        "KOLKLENGTE": "lengte"},
    snap_to_branches=snap_to_branches
    )

sluices_gdf.set_index("code", inplace=True)
sluices_gdf.replace({'0': None, 0: None}, inplace=True)
sluices_gdf.loc[sluices_gdf["lengte"].isnull(), "lengte"] = 20
sluices_gdf.loc[sluices_gdf["aantal_slu"].isnull(), "aantal_slu"] = 1
sluices_gdf["breedteopening"] = sluices_gdf.apply(
    (lambda x: principe_profielen_df.loc[x["branch_id"]][
        "bottomwidth"] if not x["breedteopening"] else x["breedteopening"]), axis=1)

sluices_gdf["hoogtebinnenonderkantbenedenstrooms"] = sluices_gdf.apply(
    (lambda x: principe_profielen_df.loc[x["branch_id"]][
        "bottomlevel"] if not x["hoogtebinnenonderkantbenedenstrooms"] else x[
            "hoogtebinnenonderkantbenedenstrooms"]), axis=1)

sluices_gdf["hoogtebinnenonderkantbovenstrooms"] = sluices_gdf.apply(
    (lambda x: principe_profielen_df.loc[x["branch_id"]][
        "bottomlevel"] if not x["hoogtebinnenonderkantbovenstrooms"] else x[
            "hoogtebinnenonderkantbovenstrooms"]), axis=1)

sluices_gdf.loc[:, "vormcode"] = "rechthoekig"
sluices_gdf.loc[:, "hoogteopening"] = 10
sluices_gdf["allowedflowdir"] = sluices_gdf.apply((lambda x: richting_mapping[x["soort"]]), axis=1)
sluices_gdf.reset_index(inplace=True)

### Toevoegen duikers en siphons
duikers en sifons worden samen met de sluizen als cuverts aan het HyDAMO object toegekend

In [11]:
culvert_groups = kwk_groups_df[kwk_groups_df["object_laag"] == "culverts"]["group_code"]

culverts_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["duikers"]),
                                    "culverts",
                                    snap_to_branches=snap_to_branches,
                                    attribute_filter={"SOORT": "tussenduiker"},
                                    column_mapping={"hoogte_a1": "hoogteopening",
                                                    "breedte": "breedteopening",
                                                    "bok_benede": "hoogtebinnenonderkantbenedenstrooms",
                                                    "bok_boven": "hoogtebinnenonderkantbovenstrooms",
                                                    "vorm": "vormcode"})

# aanmaken culverts die niet op branch liggen, maar wel horen bij een groep
grouped_culverts_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["duikers"]),
                                    "culverts",
                                    attribute_filter={"SOORT": "tussenduiker"},
                                    column_mapping={"hoogte_a1": "hoogteopening",
                                                    "breedte": "breedteopening",
                                                    "bok_benede": "hoogtebinnenonderkantbenedenstrooms",
                                                    "bok_boven": "hoogtebinnenonderkantbovenstrooms",
                                                    "vorm": "vormcode"})

grouped_culverts_gdf = grouped_culverts_gdf.loc[
    grouped_culverts_gdf["code"].isin(culvert_groups.index)
    ]

grouped_culverts_gdf = grouped_culverts_gdf.loc[
    ~grouped_culverts_gdf["code"].isin(culverts_gdf["code"])
    ]

siphons_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["sifons"]),
                                   "culverts",
                                   snap_to_branches=snap_to_branches,
                                   column_mapping={"hoogte": "hoogteopening",
                                                   "breedte": "breedteopening",
                                                   "bok_benede":"hoogtebinnenonderkantbenedenstrooms",
                                                   "bok_boven":"hoogtebinnenonderkantbovenstrooms",
                                                   "vorm":"vormcode"})

gdf = gpd.GeoDataFrame(pd.concat([
    sluices_gdf, culverts_gdf, grouped_culverts_gdf, siphons_gdf
    ], ignore_index=True))

gdf.loc[gdf['vormcode'].isnull(), 'vormcode'] = 'onbekend'
gdf.loc[:, 'vormcode'] = gdf.apply((lambda x: vorm_mapping[x['vormcode'].lower()]), axis=1)

gdf["intreeverlies"] = 0.7
gdf["uittreeverlies"] = 0.7
gdf["ruwheidswaarde"] = 70
gdf["ruwheidstypecode"] = 4

hydamo.culverts.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.culverts.snap_to_branch(hydamo.branches, snap_method="ends", maxdist=25)
hydamo.culverts.dropna(axis=0, inplace=True, subset=["branch_offset"])

hydamo.culverts.loc[hydamo.culverts["lengte"].isna(), "lengte"] = 20
hydamo.culverts.loc[hydamo.culverts["allowedflowdir"].isna(), "allowedflowdir"] = "both"

### Toevoegen stuwen

In [12]:
stuwen_ignore = ["KST0338"]

gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["stuwen"]),
                           "weirs",
                           column_mapping={"kstident": "code",
                                           "kstsoort": "soortstuwcode",
                                           "kstkrubr": "laagstedoorstroombreedte",
                                           "kstmikho": "laagstedoorstroomhoogte",
                                           "kstregel": "soortregelbaarheidcode"})

gdf = gdf.loc[~gdf["code"].isin(stuwen_ignore)]

gdf["afvoercoefficient"] = 1

hydamo.weirs.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.weirs.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=0.5)
hydamo.weirs.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Kunstwerken opruimen

Opruimen in de volgende volgorde.
- Als er op een tak een brug binnen een rekenpuntafstand van een duiker ligt, dan verwijderen we die
- Als er op een tak een duiker binnen een rekenpuntafstand van een gemaal ligt, dan verwijderen we die

In [13]:
gdf = hydamo.bridges.loc[
        hydamo.bridges.apply(
        hydrotools.filter_to_other_object,
        args=(hydamo.culverts, rekenpunt_afstand),
        axis=1)
    ]
hydamo.bridges.set_data(gdf, index_col="code", check_columns=True, check_geotype=False)

gdf = hydamo.culverts.loc[
        hydamo.culverts.apply(
        hydrotools.filter_to_other_object,
        args=(hydamo.gemalen, rekenpunt_afstand),
        axis=1)
    ]
hydamo.culverts.set_data(gdf, index_col="code", check_columns=True, check_geotype=False)

### Toevoegen afsluitmiddelen

In [14]:
gdf = pd.DataFrame({"code": [],
                    "soortafsluitmiddelcode": [],
                    "codegerelateerdobject": []})

hydamo.afsluitmiddel.set_data(gdf, index_col="code")

### Toevoegen sturing

In [15]:
gpg_gdf["bovenmarge"] = gpg_gdf["streefwaarde"] + 0.05
gpg_gdf["ondermarge"] = gpg_gdf["streefwaarde"] - 0.05
gpg_gdf["doelvariabelecode"] = 1

hydamo.sturing.set_data(gpg_gdf, index_col="code")

### Exporteren

Om het resultaat te beoordelen exporteren we alles naar shape-files. We slaan het hydamo object ook op als "pickle", zodat we bovenstaande stappen niet elke keer hoeven te herhalen

In [16]:
hydrotools.export_shapes(hydamo, path=Path(r"./hydamo_shp/boezemmodel_v4"))
hydrotools.save_model(hydamo, file_name=Path(r"./hydamo_model/boezemmodel_v4.pickle"))


## Converteren naar DFM

### Aanmaken dfm-klasse

In [17]:
dfmmodel = DFlowFMModel()

### Inlezen kunstwerken

In [18]:
dfmmodel.structures.io.weirs_from_hydamo(hydamo.weirs,
                                         yz_profiles=hydamo.crosssections,
                                         parametrised_profiles=hydamo.parametrised_profiles)

dfmmodel.structures.io.culverts_from_hydamo(hydamo.culverts,
                                            hydamo.afsluitmiddel)

dfmmodel.structures.io.bridges_from_hydamo(hydamo.bridges,
                                           yz_profiles=hydamo.crosssections,
                                           parametrised_profiles=hydamo.parametrised_profiles)

dfmmodel.structures.io.orifices_from_hydamo(hydamo.orifices)

dfmmodel.structures.io.pumps_from_hydamo(pompen=hydamo.pumps,
                                         sturing=hydamo.sturing,
                                         gemalen=hydamo.gemalen)

for key, value in dfmmodel.structures.culverts.items():
    value["allowedFlowDir"] = hydamo.culverts.loc[key]["allowedflowdir"]

2021-04-14 14:24:37,421 - delft3dfmpy.converters.hydamo_to_dflowfm - hydamo_to_dflowfm - INFO - Currently only simple weirs can be applied. From Hydamo the attributes 'laagstedoorstroomhoogte' and 'kruinbreedte' are used to define the weir dimensions.


### aanmaken compound structures

In [19]:
cmpnd_ids = [group for group, _ in kwk_groups_df.groupby("group_code")]
cmpnd_list = [df.index.to_list() for _, df in kwk_groups_df.groupby("group_code")]
dfmmodel.structures.io.compound_structures(cmpnd_ids, cmpnd_list)

2021-04-14 14:24:40,485 - delft3dfmpy.converters.hydamo_to_dflowfm - hydamo_to_dflowfm - WARNING - Structures of not on the same branche. Moving structure KDU004547 to branch OAF010059.
2021-04-14 14:24:40,486 - delft3dfmpy.converters.hydamo_to_dflowfm - hydamo_to_dflowfm - WARNING - Structures of not on the same branche. Moving structure KDU007396 to branch OAF006569.
2021-04-14 14:24:40,487 - delft3dfmpy.converters.hydamo_to_dflowfm - hydamo_to_dflowfm - WARNING - Structures of not on the same branche. Moving structure KDU010739 to branch OAF006572.


### Aanmaken 1D netwerk
We maken een 1D model met een rekenpuntafstand van 250m

In [20]:
dfmmodel.network.set_branches(hydamo.branches)
dfmmodel.network.generate_1dnetwork(one_d_mesh_distance=rekenpunt_afstand, seperate_structures=True)

2021-04-14 14:24:40,704 - delft3dfmpy.core.dfm - dfm - WARNING - Some structures are not linked to a branch.
2021-04-14 14:24:40,714 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000030 at: [0.0, 2.5425, 325.6167677365957], due to the structures at [-0.001, 0.1, 4.985, 325.6177677365957].
2021-04-14 14:24:40,716 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000036 at: [0.0, 114.3385, 265.59291912824824], due to the structures at [-0.001, 0.1, 228.577, 265.5939191282482].
2021-04-14 14:24:40,723 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000072 at: [0.0, 69.007, 118.41050377639182], due to the structures at [-0.001, 26.166, 111.848, 118.41150377639183].
2021-04-14 14:24:40,729 - delft3dfmpy.core.dfm - dfm - WARNING - Structures KDU005807, KDU007522, KDU000002, KDU010738, KDU006133, KDU034538, KDU034537, KDU007397, KDU005977, KDU005997, KDU010737 have the same location.
2021-04-14 14:24:40,731 - delft3dfmpy.cor

2021-04-14 14:24:40,838 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000472 at: [0.0, 470.366, 591.531, 637.4504582056666], due to the structures at [-0.001, 389.995, 550.737, 632.325, 637.4514582056665].
2021-04-14 14:24:40,840 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000476 at: [0.0, 175.75150000000002, 327.46950000000004, 437.765, 448.5770101944816], due to the structures at [-0.001, 128.203, 223.3, 431.639, 443.891, 448.5780101944816].
2021-04-14 14:24:40,847 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000518 at: [0.0, 93.95949999999999, 188.889, 250.59637707770685], due to the structures at [-0.001, 56.693, 131.226, 246.552, 250.59737707770685].
2021-04-14 14:24:40,850 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000519 at: [0.0, 383.943, 510.044, 664.1678159165339], due to the structures at [-0.001, 327.769, 440.117, 579.971, 664.1688159165338].
2021-04-14 14:24:40,853 - d

2021-04-14 14:24:40,962 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000953 at: [0.0, 147.679, 1103.1754838585923], due to the structures at [-0.001, 22.936, 272.422, 1103.1764838585923].
2021-04-14 14:24:40,964 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000954 at: [0.0, 3.4284999999999997, 420.3588907370588], due to the structures at [-0.001, 0.1, 6.757, 411.267, 420.3598907370588].
2021-04-14 14:24:40,969 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000987 at: [0.0, 2.3445, 95.9305, 202.635, 231.73, 299.3143180435486], due to the structures at [-0.001, 0.1, 4.589, 187.272, 217.998, 245.462, 299.3153180435486].
2021-04-14 14:24:40,971 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000991 at: [0.0, 158.313, 508.31002087104315], due to the structures at [-0.001, 127.962, 188.664, 508.3110208710431].
2021-04-14 14:24:40,974 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on b

2021-04-14 14:24:41,089 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001477 at: [0.0, 147.1095, 236.95799999999997, 388.9735, 1573.6548055944215], due to the structures at [-0.001, 121.71, 172.509, 301.407, 476.54, 687.001, 1014.608, 1573.6558055944215].
2021-04-14 14:24:41,092 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001495 at: [0.0, 181.22196999919802, 352.2889399983961], due to the structures at [-0.001, 10.255, 352.18893999839605, 352.28993999839605].
2021-04-14 14:24:41,095 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001496 at: [0.0, 3.1224999999999996, 386.50618865746685, 499.80837731493375], due to the structures at [-0.001, 0.1, 6.145, 273.304, 499.7083773149337, 499.8093773149337].
2021-04-14 14:24:41,097 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001498 at: [0.0, 668.9275, 834.3555, 1192.956195453473], due to the structures at [-0.001, 583.207, 754.648, 914.063, 1192

2021-04-14 14:24:41,218 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF002120 at: [0.0, 6.0495, 253.51471807261692], due to the structures at [-0.001, 0.1, 11.999, 253.51571807261692].
2021-04-14 14:24:41,221 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF002125 at: [0.0, 11.654, 104.3425, 263.0285, 441.7445, 626.142, 792.9720827694708], due to the structures at [-0.001, 0.1, 23.208, 185.477, 340.58, 542.909, 709.375, 792.9730827694708].
2021-04-14 14:24:41,223 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF002126 at: [0.0, 116.01050000000001, 204.24421656437076], due to the structures at [-0.001, 41.829, 190.192, 204.24521656437076].
2021-04-14 14:24:41,226 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF002141 at: [0.0, 379.8091879516572, 436.8433759033144], due to the structures at [-0.001, 322.875, 436.74337590331436, 436.84437590331436].
2021-04-14 14:24:41,229 - delft3dfmpy.core.dfm - d

...

2021-04-14 14:24:41,679 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF004345 at: [0.0, 80.8425, 173.0855, 282.0042503960714], due to the structures at [-0.001, 23.506, 138.179, 207.992, 282.0052503960714].
2021-04-14 14:24:41,684 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF004370 at: [0.0, 128.5985, 257.43243757431196], due to the structures at [-0.001, 3.939, 253.258, 257.43343757431194].
2021-04-14 14:24:41,691 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF004440 at: [0.0, 561.9735000000001, 758.489, 935.296, 1007.7239999999999, 1869.4155, 1976.0475000000001, 2049.4624418022827], due to the structures at [-0.001, 512.501, 611.446, 905.532, 965.06, 1050.388, 1812.016, 1926.815, 2025.28, 2049.463441802283].
2021-04-14 14:24:41,696 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF004454 at: [0.0, 378.1905, 573.4680000000001, 734.4025, 867.8, 1028.644, 1157.3200000000002, 1263.2645, 1384.61

2021-04-14 14:24:41,807 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF004994 at: [0.0, 32.0955, 309.71468613359576], due to the structures at [-0.001, 14.113, 50.078, 309.71568613359574].
2021-04-14 14:24:41,809 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005010 at: [0.0, 380.4355, 875.8274129085987], due to the structures at [-0.001, 145.533, 373.828, 387.043, 875.8284129085987].
2021-04-14 14:24:41,813 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005035 at: [0.0, 46.27988419066479, 55.815768381329576], due to the structures at [-0.001, 36.844, 55.715768381329575, 55.816768381329574].
2021-04-14 14:24:41,815 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005054 at: [0.0, 252.89800000000002, 331.56768675487234], due to the structures at [-0.001, 185.434, 320.362, 331.5686867548723].
2021-04-14 14:24:41,818 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005063 at

2021-04-14 14:24:41,905 - delft3dfmpy.core.dfm - dfm - WARNING - Structures KDU037775, KDU408240 have the same location.
2021-04-14 14:24:41,907 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005403 at: [0.0, 281.1585, 303.001, 362.14750000000004, 435.36882518616227], due to the structures at [-0.001, 262.99, 299.327, 306.675, 417.62, 435.36982518616225].
2021-04-14 14:24:41,910 - delft3dfmpy.core.dfm - dfm - WARNING - Structures KDU007534, KDU007564 have the same location.
2021-04-14 14:24:41,913 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005470 at: [0.0, 14.106218069140454, 19.62243613828091], due to the structures at [-0.001, 8.69, 19.52243613828091, 19.623436138280912].
2021-04-14 14:24:41,915 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005485 at: [0.0, 3.7725, 14.946571513247344], due to the structures at [-0.001, 0.1, 7.445, 14.947571513247343].
2021-04-14 14:24:41,917 - delft3dfmpy.core.dfm - dfm - IN

2021-04-14 14:24:42,026 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF006037 at: [0.0, 14.134500000000001, 41.251, 209.07849881394708, 363.9239976278942], due to the structures at [-0.001, 0.1, 28.169, 54.333, 363.8239976278942, 363.9249976278942].
2021-04-14 14:24:42,029 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF006039 at: [0.0, 271.013, 369.0380509026949], due to the structures at [-0.001, 191.073, 350.953, 369.0390509026949].
2021-04-14 14:24:42,031 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF006062 at: [0.0, 44.992999999999995, 104.06565285114873], due to the structures at [-0.001, 0.1, 89.886, 104.06665285114873].
2021-04-14 14:24:42,039 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF006300 at: [0.0, 9.6095, 102.84349743306578], due to the structures at [-0.001, 0.1, 19.119, 102.84449743306578].
2021-04-14 14:24:42,041 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes o

2021-04-14 14:24:42,169 - delft3dfmpy.core.dfm - dfm - WARNING - Structures KDU006648, KDU037752, KDU037751 have the same location.
2021-04-14 14:24:42,171 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF010438 at: [0.0, 18.0825, 98.39608837284028], due to the structures at [-0.001, 0.1, 36.065, 98.39708837284029].
2021-04-14 14:24:42,173 - delft3dfmpy.core.dfm - dfm - WARNING - Structures KDU010744, KDU004650, KDU010732, KDU004855, KDU010740, KDU002643, KDU010731, KDU010754, KDU010733, KDU010734, KDU007758, KDU004853, KDU010761, KDU010756, KDU001858, KDU004619, KDU010750, KDU010742, KDU003875, KDU010774, KDU010741, KDU010749, KDU010760, KDU004599, KDU010767, KDU010752, KDU010751, KDU010753, KDU004854, KDU005268, KDU002644, KDU004332, KDU004603, KDU004643, KDU004667, KDU004718, KDU004773, KDU010747, KDU010748, KDU010757, KDU010758, KDU010759, KDU010763, KDU010764, KDU010765, KDU010743, KDU010755, KDU010745, KDU010746, KDU010729, KDU010730 have the same location.


### Toevoegen cross-sections

Toevoegen HyDAMO profielen

In [21]:
dfmmodel.crosssections.io.from_hydamo(
    dwarsprofielen=hydamo.crosssections,
    parametrised=hydamo.parametrised_profiles,
    branches=hydamo.branches
)

2021-04-14 14:24:44,501 - delft3dfmpy.io.dfmreader - dfmreader - INFO - Before adding the number of branches without cross section is: 2508.
2021-04-14 14:24:55,168 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'dwarsprofielen' the number of branches without cross section is: 926.
No parametrised crossections available for branches.
2021-04-14 14:24:55,175 - delft3dfmpy.io.dfmreader - dfmreader - INFO - Before adding the number of structures without cross section is: 432.
2021-04-14 14:24:55,200 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'dwarsprofielen' the number of branches without cross section is: 432.
2021-04-14 14:24:55,240 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'normgeparametriseerd' the number of structures without cross section is: 0.


# Toevoegen profielen voor Lauwersmeer uit bathymetrie

We halen profielen uit de GeoTiff met bathymetrie hoogten.

In [22]:
bathymetrie_raster = bathymetrie.joinpath(bathymetriebestanden['hoogte_raster'])
bathymetrie_zones = bathymetrie.joinpath(bathymetriebestanden['zones'])

stats = ['min']
stats += ['percentile_{}'.format(perc) for perc in range(1, 5, 2)]
stats += ['percentile_{}'.format(perc) for perc in range(5, 20, 5)]
stats += ['percentile_{}'.format(perc) for perc in range(20, 80, 10)]
stats += ['percentile_{}'.format(perc) for perc in range(80, 100, 2)]
stats += ['max']

gdf = gpd.read_file(bathymetrie_zones)

with rasterio.open(bathymetrie_raster, "r") as src:
    profile = src.profile
    raster_data = src.read(1)
    affine = src.transform
    cell_area = profile['transform'][0]**2

raster_stats = zonal_stats(gdf,
                           raster_data,
                           affine=affine,
                           stats=stats,
                           nodata=profile["nodata"],
                           raster_out=True)

data = {}

for idx, (_, row) in enumerate(gdf.iterrows()):
    ident = row["ID"]
    branches = hydamo.branches.loc[
         hydamo.branches["geometry"].centroid.within(row["geometry"])
        ].index
    length = hydamo.branches.loc[branches]["geometry"].length.sum()
    data[ident] = {}
    for stat in stats:
        data[ident]["z"] = [raster_stats[idx][stat] for stat in stats]
        data[ident]['area'] = [
            np.sum(
                raster_stats[idx]['mini_raster_array'] <= raster_stats[idx][stat]
                ) * cell_area for stat in stats
            ]

    data[ident]['lenght'] = [item/length for item in data[ident]['area']]

    z = list(reversed(data[ident]["z"])) + data[ident]["z"]
    y = list(
        -(np.array(data[ident]["lenght"]) / 2)[::-1]
        ) + list(np.array(data[ident]["lenght"]) / 2)
    yz = np.array(list(zip(y, z)))
    mask = [True] + [not(item.all()) for item in np.equal(yz[1:], np.array(yz[:-1]))]
    yz = np.array([item for idx, item in enumerate(yz) if mask[idx]])

    definition = f"PRO_{ident}"
    dfmmodel.crosssections.add_yz_definition(yz=yz,
                                             thalweg=0,
                                             roughnesstype='Manning',
                                             roughnessvalue=0.035,
                                             name=definition)

    for branch in branches:
        chainage = hydamo.branches.loc[branch]["geometry"].length/2
        dfmmodel.crosssections.add_crosssection_location(branch,
                                                         chainage,
                                                         definition=definition)

Toevoegen principeprofielen op takken die nog niet over een profiel beschikken

In [23]:
if len(dfmmodel.crosssections.get_branches_without_crosssection()) > 0:
    print("adding trapezium profiles on branches with missing crosssections.")
    dfmmodel = hydrotools.add_trapeziums(dfmmodel, principe_profielen_df)

adding trapezium profiles on branches with missing crosssections.


### Toevoegen initiele condities

In [24]:
gdf = gpd.GeoDataFrame.from_file(beheerregister.joinpath(invoerbestanden["peilgebieden"]))
gdf = gdf.explode()
gdf["INDEX"] = ""

for index, df in gdf.groupby("GPGIDENT"):
    for idx, (_, row) in enumerate(df.iterrows()):
        if len(df) > 1:
            gdf.loc[row.name, 'INDEX'] = f"{index}_{idx + 1:02d}"
    else:
        gdf.loc[row.name, 'INDEX'] = index

gdf.set_index('INDEX', inplace=True)
gdf["waterlevel"] = gdf["OPVAFWWP"] + 0.2
dfmmodel.external_forcings.initial_waterlevel_polygons = gdf

### Toevoegen randvoorwaarden

In [25]:
rvw_df = pd.read_excel(excelbestanden.joinpath(modelbestanden["randvoorwaarden"]))

for _,row in rvw_df.iterrows():
    branch_id = row["BRANCH_ID"]
    pt = hydamo.branches.loc[branch_id]["geometry"].coords[int(row["COORD"])]
    series = pd.Series(data=[0.0, 0.0],
                       index=[pd.Timestamp("2000-01-01"),
                              pd.Timestamp("2100-01-01")]
                       )
    dfmmodel.external_forcings.add_boundary_condition(name=f"BND_{branch_id}",
                                                      pt=pt,
                                                      bctype=row["TYPE"],
                                                      series=series)

### Wegschrijven model

In [26]:
dfmmodel.mdu_parameters["refdate"] = 20000101
dfmmodel.mdu_parameters["tstart"] = 0.0 * 3600
dfmmodel.mdu_parameters["tstop"] = 0.5 * 3600
dfmmodel.mdu_parameters["hisinterval"] = "120. 0. 0."
dfmmodel.mdu_parameters["cflmax"] = 0.7

dfmmodel.dimr_path = dimr_path
fm_writer = DFlowFMWriter(dfmmodel, output_dir=r"modellen\boezemmodel", name="boezemmodel")

fm_writer.objects_to_ldb()
fm_writer.write_all()

if Path(r"modellen\boezemmodel\fm\boezemmodel.mdu").exists():
    print("Model is weggeschreven")
else:
    print("Er is geen model geschreven. Waarschijnlijk is iets fout gegaan")

Model is weggeschreven


## Importeren in de D-Hydro suite

Open het model nu in D-Hydro:
1. Open een "Empty Project"
2. In de "Home" ribbon, ga naar "Import" en selecteer "Flow Flexible Mesh Model"
3. Selecteer het bestand "boezemmodel.mdu" in ".\dfm_model\fm\boezemmodel.mdu"
4. Wacht tot het model is geimporteerd.....

Het geimporteerde model is nu zichtbaar in de D-Hydro suite
<img src="png/boezemmodel.png"/>